In [4]:
from systemclass import SNAIL
import numpy as np

In [5]:
EJ = 90*2*np.pi
EC = 0.177*2*np.pi
beta = 0.33
ng = 0
ncut = 300
flux = 0.48
N = 3
dims = 30

In [6]:
snail = SNAIL(EJ,EC,beta,ng,ncut,flux,N,dims)
Hs , charge_op = snail.spectrum_charge()

In [7]:
def find_optimal_k(A, B, D):
    # Define a large initial minimum difference
    min_diff = float('inf')
    optimal_k = None
    
    # Iterate over a range of possible k values
    # The range can be adjusted based on expected size of k or other insights you have about your problem
    for k in range(-1000, 1000):
        # Calculate the difference for this value of k
        diff = abs(A - (B + k * D))
        
        # If this is the smallest difference we've found so far, update min_diff and optimal_k
        if diff < min_diff:
            min_diff = diff
            optimal_k = k
            
    return optimal_k
# Function to calculate overlap (you might use inner product, fidelity, etc.)
def calculate_overlap(state1, state2):
    return abs((state1.dag() * state2)[0,0])**2

In [8]:
import qutip as qt
def floquet_spec(omega,A):
    beta = 0.15
    N = 3
    Ej = 90*2*np.pi
    Ec = 0.177*2*np.pi
    phi_ex = 0.41340478058712055
    H0,Hc = SNAIL_H(phi_ex,beta,N,Ej,Ec)
    energies,U=np.linalg.eig(H0)
    energies = energies - energies[0]*np.ones(len(energies))
    energies = energies/2/np.pi
    # U_dag = np.conjugate(U.transpose())
    # U_dag@Hc@U
    energies = np.sort(energies)
    omega_s = energies[1]
    H0 = Qobj(H0)
    Hc = Qobj(Hc)
    args = {'w': omega}
    T = (2*np.pi)/omega
    H = [H0, [Hc, lambda t, args: A*np.cos(args['w']*t)]] 
    options = Options(nsteps=1000000)  # Increasing nsteps to 10000, adjust as needed
    f_modes, f_energies = floquet_modes(H, T, args, False, options=options) 
    
    # Define your system dimension
    sdim = 20
    floquet_states = f_modes
    # Create your basis states
    basis_states = [qt.basis(sdim, i) for i in range(3)]
    # Initialize a list to hold the indices of the Floquet states with the maximum overlap for each basis state
    max_overlap_indices = [-1] * 3
    max_overlaps = [0] * 3
    # Loop over each Floquet state
    for f_index, f_state in enumerate(floquet_states):

        # Loop over each basis state
        for b_index, b_state in enumerate(basis_states):

            # Calculate the overlap
            overlap = calculate_overlap(f_state, b_state)

            # If this is the maximum overlap for this basis state, store the index
            if overlap > max_overlaps[b_index]:
                max_overlaps[b_index] = overlap
                max_overlap_indices[b_index] = f_index
    energy01 = f_energies[max_overlap_indices[1]]/2/np.pi-f_energies[max_overlap_indices[0]]/2/np.pi
    k = find_optimal_k(omega_s, energy01, omega/2/np.pi)
    energy01 =energy01+k*omega/2/np.pi
    omega_ss = energy01-omega_s
    energy02 = f_energies[max_overlap_indices[2]]/2/np.pi-f_energies[max_overlap_indices[0]]/2/np.pi
    k = find_optimal_k(2*omega_s, energy02, omega/2/np.pi)
    energy02 =energy02+k*omega/2/np.pi
    anh = energy02 - energy01 - energy01
    if (max_overlaps<(np.sqrt(2)/2*np.ones(3))).all():
        omega_ss = 1000
        anh = 1000
    return np.abs(omega_ss),anh